In [1]:
from tools.extract_sample_ids import get_sampled_ids
import json, re, os, sys
import pandas as pd
from os import path as osp
import openai
from tqdm import tqdm
from configs.openai_key import peiqi_key

openai.api_base = "https://api.chatanywhere.cn/v1"
openai.api_key = peiqi_key

# 得到小规模样本的id并保存
sample_ids = get_sampled_ids('baichuan-oreilly-sample',
                             'baichuan-13b-chat')
with open("/mnt/mfs/opsgpt/evaluation/ops-cert-eval/subset_ids.json", 'w') as f:
    json.dump(list(sample_ids), f, indent=4)

# 获得题目
with open("/mnt/mfs/opsgpt/evaluation/ops-cert-eval/v2/all.json") as f:
    data = json.load(f)
sample_data = [
    q for q in data if q['id'] in sample_ids
]


In [14]:
model_list = [
    "text-curie-001",
    "text-babbage-001",
    "text-ada-001",
    "davinci"
]

In [2]:
_hint = """
你是一个英译汉的翻译机器人。你需要将用户发来的选择题翻译成中文，你的回答只能有三行，第一行是题面的翻译，第二行是选项的翻译，第三行是类别的翻译，其他不要给出任何多余信息，下面是一个答案的样例：
---
您购买的移动设备允许您使用多指手势与设备交互。该设备最有可能使用什么类型的触摸屏技术？
['电容式', '红外线', 'SAW', '电阻式']
移动设备
---
"""

def get_translation(problem, choices, topic):
    prompt = f"{problem}\n{choices}\n{topic}"
    messages = [
        {'role': 'system', 'content': _hint},
        {'role': 'user', 'content': prompt}
    ]
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo", 
        messages=messages, 
        temperature=0)
    answer = response['choices'][0]['message']['content']
    return answer

In [8]:
with open("/mnt/mfs/opsgpt/evaluation/translated/sample.json", 'r') as f:
    data = json.load(f)



In [15]:
finished = set([d['id'] for d in data])
supplement = []
failed = []


In [16]:
for q in sample_data:
    if q['id'] not in finished:
        ans = get_translation(q['question'], q['choices'], q['topic'])
        try:
            result = ans.split('\n')
            if len(result) != 3:
                failed.append((q, ans))
                continue
            question, choices, topic = result
            q['question'] = question
            q['choices'] = choices
            q['topic'] = topic
            supplement.append(q)
            if len(supplement) % 10 == 0:
                print(len(supplement))
        except:
            print(q['id'])
            print(ans.split('\n'))

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160


In [17]:
data += supplement
len(data)

4421

In [18]:
with open("/mnt/mfs/opsgpt/evaluation/translated/sample2.json", 'w') as f:
    json.dump(data, f, indent=4)


In [ ]:
for q in data:
    assert type(q['question']) is str
    if isinstance(q['choice'], str):
        pass
    elif isinstance(q['choice'], list):
        assert(q['choice'])

In [6]:
finished

{'9781119734505-A0064',
 '9780137601066-B0006',
 '9781119787631-A0195',
 '9781119513124-A0161',
 '9780136600992-A0021',
 '9780136600992-A0086',
 '9780136634829-A0077',
 '9781119892458-A0127',
 '9781119734505-B0134',
 '9781119879619-A0102',
 '9780136733867-A0052',
 '9781119878940-A0259',
 '9780137658237-A0249',
 '9781119845256-A0075',
 '9781119807308-A0292',
 '9781119807308-B0071',
 '9781119845256-A0078',
 '9781119823810-A0037',
 '9781119854982-A0065',
 '9781119909378-A0057',
 '9781119854982-A0146',
 '9780137601066-A0066',
 '9780136755562-B0024',
 '9781119909408-A0582',
 '9781119862642-A0650',
 '9781835080399-A0159',
 '9781119879619-A0409',
 '9781119845256-A0064',
 '9781119823810-A0079',
 '9781119416920-A0562',
 '9780134307183-B0026',
 '9780138217464-A0092',
 '9781119810865-B0042',
 '9781835080399-A0150',
 '9781119807308-B0143',
 '9781394186921-A0125',
 '9781394182909-A0057',
 '9781119909378-A0021',
 '9780136575160-A0049',
 '9780134608938-B0009',
 '9781119909408-A0737',
 '9780136733867-

In [7]:
import json
from tqdm import tqdm
failed = []
translated = []
finished = []
with open("/mnt/mfs/opsgpt/evaluation/translated/sample.json", 'r') as f:
    data = json.load(f)
finished = set([d['id'] for d in data])
print(len(finished))
idx = 0
for q in tqdm(sample_data):
    if q['id'] in finished:
        continue
    try:
        ans = get_translation(q['question'], q['choices'], q['topic'])
        question, choices, topic = ans.split('\n')
        q['question'] = question
        q['choices'] = eval(choices)
        q['topic'] = topic
        translated.append(q)
        if idx % 100 == 0:
            with open("/mnt/mfs/opsgpt/evaluation/translated/sample.json", 'w') as f:
                json.dump(translated, f)
        idx += 1
    except:
        # print(q['id'], "failed!")
        failed.append(q['id'])

4201


  0%|          | 15/4954 [00:36<3:47:53,  2.77s/it]

In [31]:
with open("/mnt/mfs/opsgpt/evaluation/translated/sample2.json", 'r') as f:
    data = json.load(f)

new_selected = []

for d in data:
    assert isinstance(d['question'], str)
    if not isinstance(d['choices'], list):
        try:
            d['choices'] = eval(d['choices'])
            assert isinstance(d['choices'], list)
        except:
            continue
    if isinstance(d['topic'], str):
        try:
            d['topic'] = eval(d['topic'])
            if isinstance(d['topic'], str):
                d['topic'] = [d['topic']]
        except:
            d['topic'] = [d['topic']]
    new_selected.append(d)

In [21]:
len(new_selected)

4323

In [32]:
cnt = 0
single_question = []
for d in new_selected:
    if 'A' in d['id']:
        cnt += 1
        single_question.append(d)

In [29]:
with open("/mnt/mfs/opsgpt/evaluation/translated/sample3.json", 'w', encoding='utf-8') as f:
    json.dump(new_selected, f, indent=4)

In [35]:
with open("/mnt/mfs/opsgpt/evaluation/translated/single_test.json", 'w', encoding='utf-8') as f:
    json.dump(single_question[3:], f, indent=4)
with open("/mnt/mfs/opsgpt/evaluation/translated/single_dev.json", 'w', encoding='utf-8') as f:
    json.dump(single_question[:3], f, indent=4)

In [37]:
single_test_ids = [d['id'] for d in single_question[3:]]
single_dev_ids = [d['id'] for d in single_question[:3]]
origin_test = [d for d in sample_data if d['id'] in single_test_ids]
origin_dev = [d for d in sample_data if d['id'] in single_dev_ids]
with open("/mnt/mfs/opsgpt/evaluation/translated/original_dev.json", "w") as f:
    json.dump(origin_dev, f, indent=4)
with open("/mnt/mfs/opsgpt/evaluation/translated/original_test.json", "w") as f:
    json.dump(origin_test, f, indent=4)

In [38]:
len(origin_test) == len(single_test_ids)

True